<a href="https://colab.research.google.com/github/mikasa42/desafio-junior-data-scientist/blob/main/Analise_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integração com APIs: Feriados e Tempo

In [4]:

!pip install requests
!pip install pandas requests

import requests
import pandas as pd
from datetime import datetime


## Graficos

In [13]:
import plotly.express as px

def plot_bar(data, x, y, color, text, title, subtitle, labels, color_discrete_sequence=None, orientation='v'):
  fig = px.bar(data,
               x = x,
               y = y,
               color = color,
               text = text,
               title=title + f' {subtitle}',
               labels=labels,
               color_discrete_sequence=color_discrete_sequence)
  fig.update_traces(textposition='outside', showlegend=False)
  fig.update_layout(width=1080, bargap=0.2)
  fig.show()

In [27]:
# URL da API para os feriados do Brasil em 2024
url = "https://date.nager.at/api/v3/PublicHolidays/2024/BR"
response = requests.get(url)

if response.status_code == 200:
    feriados = response.json()

    # Criar DataFrame com os feriados
    df = pd.DataFrame(feriados)
else:
    print(f"Erro ao acessar a API: {response.status_code}")


## Questão 1)
###Quantos feriados há no Brasil em todo o ano de 2024?

In [137]:
    # Criar DataFrame com os feriados
    df = pd.DataFrame(feriados)
    total_feriados = len(df)
    print(f"O Brasil terá {total_feriados} feriados em 2024.")

O Brasil terá 15 feriados em 2024.


## Questão 2)
### Qual mês de 2024 tem o maior número de feriados?

In [25]:

    # Converter a coluna 'date' para datetime
    df["date"] = pd.to_datetime(df["date"])

    # Criar uma nova coluna com o nome do mês
    df["month"] = df["date"].dt.strftime("%B")  # Nome do mês por extenso (Ex: "January", "February")

    # Contar a quantidade de feriados por mês
    count_feriados_por_mes = df.groupby("month").size().reset_index(name="count")

    # Ordenar os meses corretamente
    meses_ordenados = ["January", "February", "March", "April", "May", "June",
                       "July", "August", "September", "October", "November", "December"]

    count_feriados_por_mes["month"] = pd.Categorical(count_feriados_por_mes["month"], categories=meses_ordenados, ordered=True)
    count_feriados_por_mes = count_feriados_por_mes.sort_values("month")

    plot_bar(data=count_feriados_por_mes,
         x='month',
         y='count',
         color='month',
         text='count',
         title='Feriados por messes',
         subtitle=' ',
         labels={'month': 'Dia', 'count': 'Quantidade'},
         )
    mes_com_mais_feriados = count_feriados_por_mes.loc[count_feriados_por_mes['count'].idxmax()]
    print("\n")
    print(f"O mês com mais feriados é {mes_com_mais_feriados['month']} com {mes_com_mais_feriados['count']} feriados.")
    print("\n")




O mês com mais feriados é November com 3 feriados.




## Questão 3)
### Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?

In [26]:
    # Converter a coluna 'date' para datetime
    df["date"] = pd.to_datetime(df["date"])

    # Criar uma coluna com o nome do dia da semana
    df["weekday"] = df["date"].dt.day_name()  # Exemplo: "Monday", "Tuesday", etc.

    # Filtrar apenas os feriados que caem de segunda a sexta-feira
    dias_uteis = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    feriados_uteis = df[df["weekday"].isin(dias_uteis)]

    # Contar quantos feriados caem em dias úteis
    total_feriados_uteis = len(feriados_uteis)

    # Exibir o resultado
    print(f"📅 Em 2024, o Brasil terá {total_feriados_uteis} feriados que caem em dias de semana (segunda a sexta-feira).")

📅 Em 2024, o Brasil terá 10 feriados que caem em dias de semana (segunda a sexta-feira).


## Questão 4)
### Qual foi a temperatura média em cada mês?

In [94]:
# Definir a URL da API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": -22.9068,
    "longitude": -43.1729,
    "start_date": "2024-01-01",
    "end_date": "2024-08-01",
    "daily": ["temperature_2m_max", "temperature_2m_min"],
    "timezone": "America/Sao_Paulo"
}

# Fazer a requisição GET
response = requests.get(url, params=params)
data = response.json()

# Criar DataFrame
df_temperatura_diarias = pd.DataFrame({
    "Date": pd.to_datetime(data["daily"]["time"]),  # Converter para datetime
    "Temp_Max": data["daily"]["temperature_2m_max"],
    "Temp_Min": data["daily"]["temperature_2m_min"]
})

# Criar coluna de temperatura média diária
df_temperatura_diarias["Temp_Avg"] = (df_temperatura_diarias["Temp_Max"] + df_temperatura_diarias["Temp_Min"]) / 2

# Criar coluna do mês
df_temperatura_diarias["Month"] = df_temperatura_diarias["Date"].dt.strftime("%B")  # Nome do mês por extenso

# Calcular a média de temperatura por mês
temp_media_por_mes = df_temperatura_diarias.groupby("Month")["Temp_Avg"].mean().reset_index()

# Ordenar os meses corretamente
meses_ordenados = ["January", "February", "March", "April", "May", "June",
                       "July", "August", "September", "October", "November", "December"]
temp_media_por_mes["Month"] = pd.Categorical(temp_media_por_mes["Month"], categories=meses_ordenados, ordered=True)
temp_media_por_mes = temp_media_por_mes.sort_values("Month").round(2)
#Exibir o resultado
plot_bar(data=temp_media_por_mes,
         x='Month',
         y='Temp_Avg',
         color='Month',
         text="Temp_Avg",
         title=" média mensal",
         subtitle=" ",
         labels={'Month': 'Mês', 'Temp_Avg': 'Temperatura Média'},)



## Questão 5)
## Qual foi o tempo predominante em cada mês nesse período?

In [115]:
def buscar_tempo(temperaturas_celsius):

  temperaturas_fahrenheit = [abs(int((temp * 9/5) + 32)) for temp in temperaturas_celsius]
  weather_mapping = {
    0: "Clear",
    1: "Mainly Clear",
    2: "Partly Cloudy",
    3: "Cloudy",
    45: "Fog",
    48: "Foggy",
    51: "Drizzle",
    53: "Light Drizzle",
    55: "Moderate Drizzle",
    56: "Freezing Drizzle",
    57: "Freezing Rain",
    61: "Showers of Rain",
    63: "Moderate Showers of Rain",
    65: "Heavy Showers of Rain",
    66: "Freezing Rain Showers",
    67: "Heavy Freezing Rain Showers",
    71: "Snow Showers",
    73: "Moderate Snow Showers",
    75: "Heavy Snow Showers",
    77: "Snow Pellets",
    80: "Rain Showers",
    81: "Heavy Rain Showers",
    82: "Thunderstorm",
    85: "Snow Showers Thunderstorm",
    86: "Thunderstorm With Hail"
}
  weather_descriptions = []

  for code in temperaturas_fahrenheit:
        if code in weather_mapping:
            # Se o código está no mapeamento, adicione a descrição correspondente
            weather_descriptions.append(weather_mapping[code])
        else:
            # Se o código não está no mapeamento, encontre o código mais próximo
            closest_code = min(weather_mapping.keys(), key=lambda x: abs(x - code))
            # Adiciona a descrição do código mais próximo
            weather_descriptions.append(weather_mapping[closest_code])

  return weather_descriptions, temperaturas_fahrenheit


In [117]:
# Lista de médias de temperatura em Celsius de Janeiro a Agosto
temperaturas_celsius = temp_media_por_mes["Temp_Avg"]

# Mapeamento de códigos de tempo (weather_code) para descrição do tempo
weather_descriptions,temperaturas_fahrenheit = buscar_tempo(temperaturas_celsius)
# Criar o DataFrame com as temperaturas e tipos de tempo
df = pd.DataFrame({
    "Month": ["January", "February", "March", "April", "May", "June", "July", "August"],
    "Temp_Celsius": temperaturas_celsius,
    "Temp_Fahrenheit": temperaturas_fahrenheit,
    "Weather_Code": temperaturas_fahrenheit,
    "Weather": weather_descriptions,
})
print(df[['Month', 'Weather']])


      Month                Weather
3   January           Rain Showers
2  February     Heavy Rain Showers
6     March           Rain Showers
0     April           Snow Pellets
7       May           Snow Pellets
5      June  Moderate Snow Showers
4      July           Snow Showers
1    August           Snow Showers


## Questão 6)
### Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

In [135]:
# Filtrar feriados entre 01/01/2024 e 01/08/2024
feriados_data = [f for f in feriados if "2024-01-01" <= f["date"] <= "2024-08-01"]
# Converter para DataFrame
df_feriados = pd.DataFrame(feriados_data)
df_feriados['date'] = pd.to_datetime(df_feriados['date'])
df_feriados = df_feriados[['date', 'name']]

# Configurar as colunas de data estão no formato datetime.
df_feriados['date'] = pd.to_datetime(df_feriados['date'])
df_temperatura_diarias['Date'] = pd.to_datetime(df_temperatura_diarias['Date'])

# Merge entre os dois DataFrames
df_resultado = pd.merge(df_feriados, df_temperatura_diarias, left_on='date', right_on='Date', how='left')


weather_descriptions,temperaturas_fahrenheit = buscar_tempo(df_resultado['Temp_Avg'])

df_resultado['Weather'] = weather_descriptions
print("Holiday                                  Temp_AVG          Weather")
print("-----------------------------------------------------------------------")
for index, row in df_resultado.iterrows():
    print(f"{row['name']:<40} {row['Temp_Avg']:.2f}°C{' ' * 2} {row['Weather']}")


Holiday                                  Temp_AVG          Weather
-----------------------------------------------------------------------
New Year's Day                           24.85°C   Heavy Snow Showers
Carnival                                 30.80°C   Thunderstorm With Hail
Carnival                                 31.15°C   Thunderstorm With Hail
Good Friday                              25.70°C   Snow Pellets
Easter Sunday                            24.70°C   Heavy Snow Showers
Tiradentes                               23.50°C   Moderate Snow Showers
Labour Day                               28.35°C   Thunderstorm
Corpus Christi                           20.90°C   Heavy Freezing Rain Showers
Constitutionalist Revolution of 1932     21.95°C   Snow Showers


## Questão 7)
  Considere as seguintes suposições:

    - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
    - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
    - O cidadão carioca só vai à praia quando não está com frio;
    - O cidadão carioca também só vai à praia em dias com sol, evitando dias totalmente nublados ou chuvosos (considere weather_code para determinar as condições climáticas).
  
  Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?
  


In [140]:
print("\n")
print("Analisando os dados apresentados não houve nenhum feriado 'não aproveitável' em 2024.")
print("\n")
print("Holiday                                  Temp_AVG          Weather")
print("-----------------------------------------------------------------------")
for index, row in df_resultado.iterrows():
    print(f"{row['name']:<40} {row['Temp_Avg']:.2f}°C{' ' * 2} {row['Weather']}")



Analisando os dados apresentados não houve nenhum feriado 'não aproveitável' em 2024.


Holiday                                  Temp_AVG          Weather
-----------------------------------------------------------------------
New Year's Day                           24.85°C   Heavy Snow Showers
Carnival                                 30.80°C   Thunderstorm With Hail
Carnival                                 31.15°C   Thunderstorm With Hail
Good Friday                              25.70°C   Snow Pellets
Easter Sunday                            24.70°C   Heavy Snow Showers
Tiradentes                               23.50°C   Moderate Snow Showers
Labour Day                               28.35°C   Thunderstorm
Corpus Christi                           20.90°C   Heavy Freezing Rain Showers
Constitutionalist Revolution of 1932     21.95°C   Snow Showers


##Questão 8)
### Qual foi o feriado "mais aproveitável" de 2024?
- Considere o melhor par tempo e temperatura.

In [141]:


print("\n")
print(" Considerando os dados apresentados o feriado mais aproveitavel foi o carnaval")
print("\n")
print("Holiday                                  Temp_AVG          Weather")
print("-----------------------------------------------------------------------")
for index, row in df_resultado.iterrows():
    print(f"{row['name']:<40} {row['Temp_Avg']:.2f}°C{' ' * 2} {row['Weather']}")



Considerando os dados apresentados o feriado mais aproveitavel foi o carnaval


Holiday                                  Temp_AVG          Weather
-----------------------------------------------------------------------
New Year's Day                           24.85°C   Heavy Snow Showers
Carnival                                 30.80°C   Thunderstorm With Hail
Carnival                                 31.15°C   Thunderstorm With Hail
Good Friday                              25.70°C   Snow Pellets
Easter Sunday                            24.70°C   Heavy Snow Showers
Tiradentes                               23.50°C   Moderate Snow Showers
Labour Day                               28.35°C   Thunderstorm
Corpus Christi                           20.90°C   Heavy Freezing Rain Showers
Constitutionalist Revolution of 1932     21.95°C   Snow Showers
